In [18]:
import requests
import time
from time import sleep
from bs4 import BeautifulSoup
import re
import pandas as pd





In [8]:
# params
coin_id = 1 # btc
currency_id = 2790 # eur;  2781 # usd 
unix_st = 946684800 # always the same, 2000-01-01-00:00:00
unix_current_date = int(time.time()) # current time in unix timestamp

url = f'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/historical?id={coin_id}&convertId={currency_id}&timeStart={unix_st}&timeEnd={unix_current_date}'

In [9]:
# get html page for historical
response = requests.get(url)

In [10]:
# beautifulSoup creation
soup = BeautifulSoup(response.text, 'lxml')



In [11]:
# extract the information regarding each day 
main_pattern= 'timeOpen(.*?)timestamp'

main_info = re.findall(main_pattern, soup.find_all('p')[0].text)[0]

# optimize text for info extraction
main_info_optimized = main_info.replace('","', ',')
main_info_optimized = main_info_optimized.replace('":"', ',')
main_info_optimized = main_info_optimized.replace('{"', ',')
main_info_optimized = main_info_optimized.replace('":', ',')
main_info_optimized = main_info_optimized.replace(',"', ',')
main_info_optimized = main_info_optimized.replace(',,', ',')

# split the content on the comas
info_split = main_info_optimized.split(',')


info_split

['',
 '2013-04-28T00:00:00.000Z',
 'timeClose',
 '2013-04-28T23:59:59.999Z',
 'timeHigh',
 '2013-04-28T18:50:02.000Z',
 'timeLow',
 '2013-04-28T20:15:02.000Z',
 'quote',
 'open',
 '103.8452705900',
 'high',
 '104.3671776249',
 'low',
 '101.3892133729',
 'close',
 '103.0086781133',
 'volume',
 '0E-14',
 'marketCap',
 '1142502669.6216133241',
 '']

## Structure of each paragraph containing information as of January 7th 2022

- pos 1 -> TimeOpen
- pos 3 -> TimeClose
- pos 5 -> timeHigh
- pos 7 -> TimeLow
- pos 10 -> OpenPrice
- pos 12 -> HighPrice
- pos 14 -> LowPrice
- pos16 -> ClosePrice
- pos 18 -> Volume
- pos 20 -> MrktCap

In [12]:
dct_ = {
    'timeOpen':[],
    'timeClose':[],
    'timeHigh': [], 
    'timeLow': [],
    'openPrice':[],
    'highPrice': [],
    'lowPrice':[],
    'closePrice':[],
    'volume':[],
    'mrktCap':[]
}
for i, val in enumerate(info_split):
    if i == 1:
        dct_['timeOpen'].append(val.replace("''", ''))
    elif i == 3:
        dct_['timeClose'].append(val.replace("''", ''))
    elif i == 5:
        dct_['timeHigh'].append(val.replace("''", ''))
    elif i == 7:
        dct_['timeLow'].append(val.replace("''", ''))
    elif i == 10:
        dct_['openPrice'].append(val.replace("''", ''))
    elif i == 12:
        dct_['highPrice'].append(val.replace("''", ''))
    elif i == 14:
        dct_['lowPrice'].append(val.replace("''", ''))
    elif i == 16:
        dct_['closePrice'].append(val.replace("''", ''))
    elif i == 18:
        dct_['volume'].append(val.replace("''", ''))
    elif i == 20:
        dct_['mrktCap'].append(val.replace("''", ''))
    

In [13]:
dct_

{'timeOpen': ['2013-04-28T00:00:00.000Z'],
 'timeClose': ['2013-04-28T23:59:59.999Z'],
 'timeHigh': ['2013-04-28T18:50:02.000Z'],
 'timeLow': ['2013-04-28T20:15:02.000Z'],
 'openPrice': ['103.8452705900'],
 'highPrice': ['104.3671776249'],
 'lowPrice': ['101.3892133729'],
 'closePrice': ['103.0086781133'],
 'volume': ['0E-14'],
 'mrktCap': ['1142502669.6216133241']}

## Testing script on full BTC historical data

In [16]:
dct_ = {
    'timeOpen':[],
    'timeClose':[],
    'timeHigh': [], 
    'timeLow': [],
    'openPrice':[],
    'highPrice': [],
    'lowPrice':[],
    'closePrice':[],
    'volume':[],
    'mrktCap':[]
}

# extract the information regarding each day 
main_pattern= 'timeOpen(.*?)timestamp'

main_info = re.findall(main_pattern, soup.find_all('p')[0].text)

for date, hist in enumerate(main_info):
    # date list pos, hist info of single event
    # optimize text for info extraction
    tmp = hist.replace('","', ',')
    tmp = tmp.replace('":"', ',')
    tmp = tmp.replace('{"', ',')
    tmp = tmp.replace('":', ',')
    tmp = tmp.replace(',"', ',')
    tmp = tmp.replace(',,', ',')

    # split the content on the comas
    tmp_split = tmp.split(',')

    
    for i, val in enumerate(tmp_split):
        if i == 1:
            dct_['timeOpen'].append(val.replace("''", ''))
        elif i == 3:
            dct_['timeClose'].append(val.replace("''", ''))
        elif i == 5:
            dct_['timeHigh'].append(val.replace("''", ''))
        elif i == 7:
            dct_['timeLow'].append(val.replace("''", ''))
        elif i == 10:
            dct_['openPrice'].append(val.replace("''", ''))
        elif i == 12:
            dct_['highPrice'].append(val.replace("''", ''))
        elif i == 14:
            dct_['lowPrice'].append(val.replace("''", ''))
        elif i == 16:
            dct_['closePrice'].append(val.replace("''", ''))
        elif i == 18:
            dct_['volume'].append(val.replace("''", ''))
        elif i == 20:
            dct_['mrktCap'].append(val.replace("''", ''))

In [19]:
df = pd.DataFrame.from_dict(dct_)
df.head()

,timeOpen,timeClose,timeHigh,timeLow,openPrice,highPrice,lowPrice,closePrice,volume,mrktCap
0,2013-04-28T00:00:00.000Z,2013-04-28T23:59:59.999Z,2013-04-28T18:50:02.000Z,2013-04-28T20:15:02.000Z,103.8452705900,104.3671776249,101.3892133729,103.0086781133,0E-14,1142502669.6216133241
1,2013-04-29T00:00:00.000Z,2013-04-29T23:59:59.999Z,2013-04-29T13:15:01.000Z,2013-04-29T05:20:01.000Z,103.1249522468,113.1303264492,102.7843829102,110.8690598191,0E-14,1230167112.4491830943
2,2013-04-30T00:00:00.000Z,2013-04-30T23:59:59.999Z,2013-04-30T08:25:02.000Z,2013-04-30T18:55:01.000Z,109.9740339087,112.2116944217,102.3751359797,106.1554910646,0E-14,1178259603.6352527875
3,2013-05-01T00:00:00.000Z,2013-05-01T23:59:59.999Z,2013-05-01T00:15:01.000Z,2013-05-01T19:55:01.000Z,105.5910057733,106.2670916056,81.8292321640,88.8711621572,0E-14,986747640.3448799757
4,2013-05-02T00:00:00.000Z,2013-05-02T23:59:59.999Z,2013-05-02T14:25:01.000Z,2013-05-02T14:30:02.000Z,88.3071532388,95.3031326156,70.0219276869,79.8315494988,0E-14,886651108.0127475529
